In [42]:
!wget https://www.dropbox.com/s/7h6kf53lpi5v39w/alice.txt -O wonderland.txt

--2019-07-23 16:53:47--  https://www.dropbox.com/s/7h6kf53lpi5v39w/alice.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7h6kf53lpi5v39w/alice.txt [following]
--2019-07-23 16:53:47--  https://www.dropbox.com/s/raw/7h6kf53lpi5v39w/alice.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucee03d101f31e6f30ceece81591.dl.dropboxusercontent.com/cd/0/inline/AlQgakvAE4UxsXoZAbH7vZ6Unwl2KxMCbpKnO3VaftA5wR6V6RycT14VCYDD9OfKbjmLpl5cEJaiP0I_EJmhU-WD5c3vFwcvs9fHoUkFgLAuVQ/file# [following]
--2019-07-23 16:53:48--  https://ucee03d101f31e6f30ceece81591.dl.dropboxusercontent.com/cd/0/inline/AlQgakvAE4UxsXoZAbH7vZ6Unwl2KxMCbpKnO3VaftA5wR6V6RycT14VCYDD9OfKbjmLpl5cEJaiP0I_EJmhU-WD5c3vFwcvs9fHoUkFgLAuVQ/file
Resolving ucee03d101f31e6

In [0]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [58]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, encoding='utf-8',errors='ignore').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163779
Total Vocab:  58


In [0]:
import string 
raw_text = raw_text.translate(str.maketrans('', '', string.punctuation))

In [48]:
raw_text

'project gutenbergs alices adventures in wonderland by lewis carroll\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever  you may copy it give it away or\nreuse it under the terms of the project gutenberg license included\nwith this ebook or online at wwwgutenbergorg\n\n\ntitle alices adventures in wonderland\n\nauthor lewis carroll\n\nposting date june 25 2008 ebook 11\nrelease date march 1994\nlast updated december 20 2011\n\nlanguage english\n\n\n start of this project gutenberg ebook alices adventures in wonderland \n\n\n\n\n\n\n\n\n\n\nalices adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 30\n\n\n\n\nchapter i down the rabbithole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank and of having nothing to do once or twice she had peeped into the\nbook her sister was reading but it had no pictures or conversations in\nit and what is the use of a book thought alice without pict

In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
from keras.preprocessing.sequence import pad_sequences

dataX = pad_sequences(dataX)
# for x in range(len(dataX)):
#   print(len(dataX[x]))

In [0]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
# model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
163679/163679 [==============================] - 486s 3ms/step - loss: 2.9466

Epoch 00001: loss improved from inf to 2.94663, saving model to weights-improvement-01-2.9466-bigger.hdf5
Epoch 2/100
163679/163679 [==============================] - 481s 3ms/step - loss: 2.7742

Epoch 00002: loss improved from 2.94663 to 2.77424, saving model to weights-improvement-02-2.7742-bigger.hdf5
Epoch 3/100
163679/163679 [==============================] - 477s 3ms/step - loss: 2.6561

Epoch 00003: loss improved from 2.77424 to 2.65614, saving model to weights-improvement-03-2.6561-bigger.hdf5
Epoch 4/100
163679/163679 [==============================] - 470s 3ms/step - loss: 2.5473

Epoch 00004: loss improved from 2.65614 to 2.54735, saving model to weights-improvement-04-2.5473-bigger.hdf5
Epoch 5/100
163679/163679 [==============================] - 469s 3ms/step - loss: 2.4595

Epoch 00005: loss improved from 2.54735 to 2.45946, saving model to weights-improvement-05-2.4595-bigger.hdf5